In [1]:
import pandas as pd
import time 
import pickle
import os
#nltk.download('punkt')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings("ignore")

In [2]:
#import pre-processed data from pickle 
#df_10= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2010_text_wo_names.pickle")
#df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2009_text_wo_names.pickle")
#df_11= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2011_text_wo_names.pickle")
#df_12= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2012_text_wo_names.pickle")
#df_13= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2013_text_wo_names.pickle")
#df_14= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2014_text_wo_names.pickle")
df_15= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2015_text_wo_names.pickle")
#df_16= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2016_text_wo_names.pickle")
#df_17= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2017_text_wo_names.pickle")
#df_18= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2018_text_wo_names.pickle")
#df_19 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2019_text_wo_names.pickle")
#df_20 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2020_text_wo_names.pickle")
#df_21 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2021_text_wo_names.pickle")
#df_22 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2022_text_wo_names_(1).pickle")
df_15

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
0,"[star, die, age]",1,0,"[Welsh, Roger]",True,"Welsh actor Roger Rees, star of Cheers and The...",18,2015,0
1,"[bear, win, award, title, role]",1,0,[Nicholas],None,"Mr Rees, who was born in Aberystwyth, Ceredigi...",18,2015,0
2,"[play, millionaire, comedy, cheers, political,...",1,0,"[Robin, Colcord, John]",True,He played millionaire Robin Colcord in 1980s c...,18,2015,0
4,"[play, love, interest, play, cheers]",1,0,[Rebecca],None,He played a love interest to Cheers' Rebecca H...,18,2015,0
5,"[tweet, sweetest, world]",2,1,[],None,"She tweeted: ""He is the sweetest man in the wo...",18,2015,0
...,...,...,...,...,...,...,...,...,...
3258196,"[one, pictured, point, crossbow, camera, wear,...",0,1,[],None,"In one, she is pictured pointing a crossbow at...",2187120,2015,1
3258197,"[chief, prosecutor, say, exchange, phone, call...",0,1,[],None,"France's chief prosecutor, Francois Molins, sa...",2187120,2015,1
3258199,"[say, record, show, interrogate, french, offic...",0,2,[],None,It says the records show she was once interrog...,2187120,2015,1
3258200,"[friend, tell, last, saw, month, ago, dinner]",0,3,"[Boumeddiene, Le]",True,A friend of Boumeddiene told Le Parisien she l...,2187120,2015,1


In [3]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [4]:
#apply function to only get rows with an absolute count 
df_15['col_type'] = df_15.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)

#remove nulls 
df_15 = df_15[df_15["col_type"].notnull()]

#DOC: number of male and female columns
df_15["col_type"].value_counts()  

0.0    621136
1.0    219804
Name: col_type, dtype: int64

**Define TFIDF Vectorizer**

In [5]:
#tfidf vectorizer
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

**Logistic Regression Classifier**

*What the LR model does-* LR estimates the probability of an instance belonging to the positive class. 

In [6]:
def logistic_regression_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10, 100],
        'solver': ['lbfgs', 'newton-cg', 'sag' 'saga'], #removed liblinear as it is for small + medium datasets & NOT for sparse data
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_clf = LogisticRegression(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    #results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Accuracy: {accuracy:.2f}")
    #print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:1000]
    low_coef = sorted_coef[-1000:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])

    #save model 
    with open('results.pkl', 'wb') as f:
        pickle.dump({'accuracy': accuracy, 'report': class_report}, f)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef, best_clf, class_report #df_probs_top

**Coefficient Analysis**

In [7]:
df_probs_15 = logistic_regression_year(df_15, 'col_type', 'pre_processed_sent')
df_probs_15


Execution time: 1220.47 seconds


(        feature      coef
 0      pregnant  8.814294
 1       harrier  7.743049
 2         hijab  7.550609
 3       handbag  7.538057
 4     pregnancy  7.139974
 ..          ...       ...
 995      gallop  2.981413
 996  antoinette  2.981280
 997      aiadmk  2.980665
 998       zuhra  2.979551
 999      titina  2.978756
 
 [1000 rows x 2 columns],
            feature      coef
 0            clone -2.772815
 1               yr -2.772969
 2            lunge -2.772971
 3    gratification -2.773645
 4        hungarian -2.774505
 ..             ...       ...
 995     knighthood -5.673168
 996       redmayne -5.690594
 997      backbench -5.752495
 998       gatherer -6.703643
 999       prostate -7.600947
 
 [1000 rows x 2 columns],
 LogisticRegression(C=10, class_weight={0: 0.3, 1: 0.7}, random_state=42,
                    solver='newton-cg'),
 '              precision    recall  f1-score   support\n\n         0.0       0.84      0.78      0.81    124413\n         1.0       0.48      0.

*Interpreting Performance*

The LR model is WAY better in terms of precision, recall, and f1-score at predicting the negative class - i.e. male. 

In [9]:
#open model performance metrics 
with open('results_15.pkl', 'rb') as f:
    results = pickle.load(f)

accuracy = results['accuracy']
report = results['report']

print(f"Accuracy: {accuracy:.2f}")
print(f"Classification report:\n{report}")

Accuracy: 0.73
Classification report:
              precision    recall  f1-score   support

         0.0       0.84      0.78      0.81    124413
         1.0       0.48      0.58      0.52     43775

    accuracy                           0.73    168188
   macro avg       0.66      0.68      0.66    168188
weighted avg       0.74      0.73      0.73    168188



In [10]:
#create DF of highest coef
highest_coef_15 = pd.DataFrame(df_probs_15[0])
highest_coef_15["coef_type"] = "highest"
highest_coef_15["year"] = 2015

#create DF of lowest lowest coef manipulation 
lowest_coef_15 = pd.DataFrame(df_probs_15[1]) 
lowest_coef_15 = lowest_coef_15.sort_values(by = ["coef"], ascending = True).reset_index(drop = True) #absolute lowest value 
lowest_coef_15["coef_type"] = "lowest" #coef type
lowest_coef_15["year"] = 2015 #year 
highest_coef_15

,feature,coef,coef_type,year
0,pregnant,8.814294,highest,2015
1,harrier,7.743049,highest,2015
2,hijab,7.550609,highest,2015
3,handbag,7.538057,highest,2015
4,pregnancy,7.139974,highest,2015
...,...,...,...,...
995,gallop,2.981413,highest,2015
996,antoinette,2.981280,highest,2015
997,aiadmk,2.980665,highest,2015
998,zuhra,2.979551,highest,2015


In [11]:
#save DF as pickle file per year 
lowest_coef_15.to_pickle('RESULTS15_coef_low.pickle')
highest_coef_15.to_pickle('RESULTS15_coef_high.pickle')

**Interpretation of coefficient results**
"winner" is one of the independent variables in the model and its coefficient value is 1.2865737872946597. This means that a one unit increase in the value of the "winner" variable will increase the log-odds of the positive class (e.g. "female" if the logistic regression model is binary and predicting gender) by the corresponding coefficient value, while holding all other variables constant.

**Interpretation of the predicted probabilities in LR:** 
The predicted probabilities of the logistic regression model tell us the probability that the input data belongs to the positive class - in this case the female class as we attributed it a value = 1 in binary log reg. Hence, for each word, we get a list of a word/feature and the probability that it is female. 

These predicted probabilities can be interpreted as the confidence level of the model in its prediction. For example, a predicted probability of 0.8 for a positive class means that the model is 80% confident that the sample belongs to the positive class. 

In [12]:
def load_pickle_files_low(directory):
    objects = []
    for filename in os.listdir(directory):
        if filename.endswith("low.pickle"):
            with open(os.path.join(directory, filename), 'rb') as file:
                obj = pickle.load(file)
                objects.append(obj)
    return objects

def load_pickle_files_high(directory):
    objects = []
    for filename in os.listdir(directory):
        if filename.endswith("high.pickle"):
            with open(os.path.join(directory, filename), 'rb') as file:
                obj = pickle.load(file)
                objects.append(obj)
    return objects

In [13]:
#stack dfs together for low and high coefs 
df_low = load_pickle_files_low(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis")
df_high = load_pickle_files_high(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis")
df_high

[           feature      coef coef_type  year
 0         pregnant  3.693514   highest  2012
 1             baby  3.158639   highest  2012
 2           female  3.043889   highest  2012
 3              rap  2.908451   highest  2012
 4          implant  2.901732   highest  2012
 ..             ...       ...       ...   ...
 995          sheet  0.265721   highest  2012
 996      whitehawk  0.265642   highest  2012
 997  heartbreaking  0.265527   highest  2012
 998         rights  0.265453   highest  2012
 999        winning  0.265396   highest  2012
 
 [1000 rows x 4 columns],
             feature      coef coef_type  year
 0          pregnant  3.947211   highest  2013
 1               rap  3.134167   highest  2013
 2         boyfriend  2.970981   highest  2013
 3              baby  2.843502   highest  2013
 4            female  2.827021   highest  2013
 ..              ...       ...       ...   ...
 995    specifically  0.263976   highest  2013
 996     provocation  0.263957   highest  20